STAT406 - Lecture 1 notes
================
Matias Salibian-Barrera
2018-08-19

#### LICENSE

These notes are released under the "Creative Commons Attribution-ShareAlike 4.0 International" license. See the **human-readable version** [here](https://creativecommons.org/licenses/by-sa/4.0/) and the **real thing** [here](https://creativecommons.org/licenses/by-sa/4.0/legalcode).

Lecture slides
--------------

The lecture slides will be here.

Tools you may find useful during the course
-------------------------------------------

-   [R](http://www.cran.r-project.org/): This is the software we will use in the course. I will assume that you are familiar with it (in particular, that you know how to write **your own functions** and **loops**). If needed, there are plenty of resources on line to learn R.
-   [RStudio](https://www.rstudio.com/products/RStudio/): The IDE (integrated development environment) of choice for R. Not necessary, but helpful.
-   [Jupyter Notebooks](https://www.datacamp.com/community/blog/jupyter-notebook-r): With these you will be able to reproduce (and play with) all the examples I use in class. Not necessary, but helpful.

Predictions using a linear model
--------------------------------

In this document we will explore (rather superficially) some challenges found when trying to estimate the forecasting properties (e.g. the mean squared prediction error) of a (linear) predictor. We will use the air-pollution data set, which I have split into a *training set* and a *test set*. The test set will be ignored when **training** our model (in the case of a linear model, "**training**" simply means "**when estimating the vector of linear regression parameters**").

If you are interested in how these sets (*training* and *test*) were constructed: I ran the following script (you do not need to do this, as I am providing both data sets to you, but you can re-create them yourself if you want to). In the code chunk below, `x.tr` denotes the training set and `x.te` the test set. 

In [ ]:
x <- read.csv('rutgers-lib-30861_CSV-1.csv')
set.seed(123)
ii <- sample(rep(1:4, each=15))
# this is the training set `pollution-train.dat`
x.tr <- x[ii != 2, ]
# this is the test set `pollution-test.dat`
x.te <- x[ii == 2, ]
# then I saved them to disk:
# write.csv(x.tr, file='pollution-train.dat', row.names=FALSE, quote=FALSE)
# write.csv(x.te, file='pollution-test.dat', row.names=FALSE, quote=FALSE)

Ignoring the above, we will now read the **training** data set from the file `pollution-train.dat`, which is available [here](pollution-train.dat), and check that it was read properly:

In [ ]:
x.tr <- read.table('pollution-train.dat', header=TRUE, sep=',')
# sanity check
head(x.tr)

The response variable is `MORT`. Our first step is to fit a linear regression model with all available predictors and look at a few diagnostic plots where everything looks fine:

In [ ]:
full <- lm(MORT ~ . , data=x.tr)
plot(full, which=1)

In [ ]:
plot(full, which=2)

We also take a look at the estimated coeficients:

In [ ]:
summary(full)

The fit appears to be routine, and reasonable (why? what did I check to come to this conclusion?).

### A new focus: prediction

This course will be primarily concerned with making (good) predictions for cases (data points) that we may have not observed yet (future predictions). This is a bit different from the focus of other Statistics courses you may have taken. You will see later in the course that what you learned in other Statistics courses (e.g. trade-offs between flexibility and stability of different models, uncertainty and standard techniques to reduce it, etc.) will prove to be critical for building good predictions.

As a simple example, in the rest of this note we will compare the quality of this model's predictions with those of a simpler (smaller) linear model with only 5 predictors. For this illustrative example, we will not worry about how these 5 explanatory variables were selected, however, this will play a **critical** role later in the course).

We now fit this **reduced** model and look at the estimated parameters and diagnostic plots

In [ ]:
reduced <- lm(MORT ~ POOR + HC + NOX + HOUS + NONW, data=x.tr)
summary(reduced)

In [ ]:
plot(reduced, which=1)

In [ ]:
plot(reduced, which=2)

Although the reduced linear model (with 5 predictors) does not seem to provide a fit as good as the one we get with full model, it is still acceptable.

In [ ]:
sum( resid(reduced)^2 )

In [ ]:
sum( resid(full)^2 )

This observation should be obvious to you, since, as you already now, a model will **always** yield a better fit to the data in terms of residual sum of squares than any of its submodels (i.e. any model using a subset of the explanatory variables). I expect you to be able to formally prove the last satement.

Our question of interest here is: "Which model produces better predictions?" In many cases one is interested in predicting future observations, i.e. predicting the response variable for data that was not available when the model / predictor was *fit* or *trained*. As we discussed in class, a reasonably fair comparison can be obtined by comparing the mean squared predictions of these two linear models on the test set, which we read into `R` as follows:

In [ ]:
x.te <- read.table('pollution-test.dat', header=TRUE, sep=',')
head(x.te)

Now compute the predicted values for the test set with both the **full** and **reduced** models:

In [ ]:
x.te$pr.full <- predict(full, newdata=x.te)  
x.te$pr.reduced <- predict(reduced, newdata=x.te)  

and compute the corresponding mean squared prediction errors:

In [ ]:
with(x.te, mean( (MORT - pr.full)^2 ))

In [ ]:
with(x.te, mean( (MORT - pr.reduced)^2 ))

Note that the reduced model (that did not fit the data as well as the full model) nevertheless produced better predictions (smaller mean squared prediction errors) on the test set.

At this point you should put on your critical / skeptical hat and wonder if this did not happen *by chance*, i.e. if this may be just an artifact of the specific training/test partition we used. The following simple experiment shows that this is not the case. It would be a **very good exercise** for you to repeat it many times (100, say) to verify my claim.

First, read the whole data and create a new training / test random split.

In [ ]:
# repeat with different partitions
x <- read.csv('rutgers-lib-30861_CSV-1.csv')
set.seed(456)
ii <- sample(rep(1:4, each=15))
x.tr <- x[ii != 2, ]
x.te <- x[ii == 2, ]

In the above code chunk, I used `x.tr` to denote the training set and `x.te` for the test set. Now, fit the full and reduced models on this new training set:

In [ ]:
full <- lm(MORT ~ . , data=x.tr)
reduced <- lm(MORT ~ POOR + HC + NOX + HOUS + NONW, data=x.tr)

Finally, estimate the mean squared prediction error of these models with their squared prediction error on the test set:

In [ ]:
x.te$pr.full <- predict(full, newdata=x.te)
x.te$pr.reduced <- predict(reduced, newdata=x.te)
with(x.te, mean( (MORT - pr.full)^2 ))

In [ ]:
with(x.te, mean( (MORT - pr.reduced)^2 ))

Note that the estimated mean squared prediction error of the reduced model is again considerably smaller than that of the full model (even though the latter always fits the training set better than the reduced one).